In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets

import altair as alt
import pandas as pd
from datetime import datetime

In [2]:
from scripts.pointcloud_db import ConePositionTable, PointCloudTable, get_run_progressive_bounding_boxes

# Exploració de dades

Tenint totes les posicions registrades del cotxe durant la run d'un rosbag i sabent en quin instant de temps un punt ha estat rebut, som capaços d'obtenir bounding boxes a temps real. És a dir, donada una posició del cotxe i el seu instant de temps, podem obtenir els punts rebuts en aquell moment. Amb altres dades que tenim també registrades podem, a més calcular la distància del cotxe al con i extreure les bounding boxes al voltant d'hipotètics cons en cada instant de temps.

# Acumulació dels 2 segons anteriors

Primerament extreurem doncs, al llarg d'una run, totes les bounding boxes formades pels punts rebuts entre l'instant de cada posició i els `delta` ms posteriors.

In [3]:
last_2000ms = get_run_progressive_bounding_boxes(position_step=50, delta=2000)

In [4]:
time_zero_last_2000ms = datetime.strptime(last_2000ms[0][2], '%Y-%m-%d %H:%M:%S.%f')
for i in range(len(last_2000ms)):
    last_2000ms[i] += (datetime.strptime(last_2000ms[i][2], '%Y-%m-%d %H:%M:%S.%f'), )

In [5]:
df_last_2000ms = pd.DataFrame(last_2000ms, columns=['size', 'dist', 'time_str', 'time'])

bars_last_2000ms = alt.Chart(df_last_2000ms).mark_bar().encode(
    x=alt.X("dist:Q", bin=True),
    y='mean(size):Q',
).properties(
    height=200,
    width=800,
)

heatmap_last_2000ms = alt.Chart(df_last_2000ms).mark_rect().encode(
    alt.X('dist:Q', bin=alt.Bin(maxbins=8)),
    alt.Color('count():Q', scale=alt.Scale(scheme='greenblue'))
).properties(
    width=800,
)

bubble_last_2000ms = alt.Chart(df_last_2000ms).mark_circle().encode(
    x='dist:Q',
    y='time:T',
    size='size:Q'
).properties(
    width=800,
)

last_2000ms_plot = alt.vconcat(bars_last_2000ms, heatmap_last_2000ms, bubble_last_2000ms)

In [6]:
last_2000ms_plot

/home/pau/.local/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

## Acumulació des de l'estat inicial

Seguidament extraiem, al llarg d'una run, totes les bounding boxes formades pels punts rebuts abans de l'instant de cada posició.

In [7]:
beggining = get_run_progressive_bounding_boxes(position_step=100)

In [8]:
time_zero_beggining = datetime.strptime(beggining[0][2], '%Y-%m-%d %H:%M:%S.%f')
for i in range(len(beggining)):
    beggining[i] += (datetime.strptime(beggining[i][2], '%Y-%m-%d %H:%M:%S.%f'), )

In [9]:
df_beggining = pd.DataFrame(beggining, columns=['size', 'dist', 'time_str', 'time'])

bars = alt.Chart(df_beggining).mark_bar().encode(
    x=alt.X("dist:Q", bin=True),
    y='mean(size):Q',
).properties(
    height=200,
    width=800,
)

heatmap = alt.Chart(df_beggining).mark_rect().encode(
    alt.X('dist:Q', bin=alt.Bin(maxbins=8)),
    alt.Color('count():Q', scale=alt.Scale(scheme='greenblue'))
).properties(
    width=800,
)

bubble = alt.Chart(df_beggining).mark_circle().encode(
    x='dist:Q',
    y='time:T',
    size='size:Q'
).properties(
    width=800,
)

beggining_plot = alt.vconcat(bars, heatmap, bubble)

In [10]:
beggining_plot

/home/pau/.local/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)